* TOC
{:toc}

## 第六章：面向对象编程 

### 第二节：魔术方法与类装饰器 

#### 1.Python中的魔术方法 

魔术方法（也称为特殊方法）是Python中的一组预定义的方法，它们以双下划线（`__`）开始和结束。这些方法允许开发者为自定义类添加“魔法”行为，使得这些类能够模拟一些内置类型的行为，或者响应内置函数和运算符的操作。魔术方法提供了一种方式，让自定义对象能够集成到Python的语言特性中。

常见的魔术方法包括：

 *  `__init__(self, ...)`: 构造函数，创建对象时调用。
 *  `__del__(self)`: 析构函数，对象被销毁时调用。
 *  `__str__(self)`: 定义对象的“非正式”或可打印的字符串表示。
 *  `__repr__(self)`: 定义对象的“正式”字符串表示，通常可以用来重建这个对象。
 *  `__len__(self)`: 使对象能够使用`len()`函数。
 *  `__getitem__(self, key)`: 使对象支持索引和切片操作。
 *  `__setitem__(self, key, value)`: 使对象支持通过索引或键赋值。
 *  `__eq__(self, other)`: 定义等于操作符的行为。
 *  `__add__(self, other)`: 定义加法操作符的行为。

示例：使用魔术方法实现一个简单的类


In [ ]:
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author

    def __str__(self):
        return f"'{self.title}' by {self.author}"

book = Book("Python 101", "John Doe")
print(book)  # 输出: 'Python 101' by John Doe


#### 2.Python中的类装饰器 

类装饰器是一种函数，它接受一个类作为参数，并返回一个类。类装饰器可以用来修改或增强类的行为。它们提供了一种灵活的方式，通过在类定义外部添加额外的功能来扩展类的功能，而不需要修改类的内部定义。

使用类装饰器的一般形式：


In [ ]:
def my_decorator(cls):
    # 修改或增强类cls
    return cls

@my_decorator
class MyClass:
    pass


示例：使用类装饰器为类添加一个新方法


In [ ]:
def add_method(cls):
    cls.new_method = lambda self: "This is a new method"
    return cls

@add_method
class MyClass:
    pass

obj = MyClass()
print(obj.new_method())  # 输出: This is a new method


在这个示例中，`add_method`装饰器为`MyClass`类添加了一个新的方法`new_method`。当创建`MyClass`的实例并调用这个新方法时，它返回了预期的字符串。

魔术方法和类装饰器都是Python面向对象编程中的高级特性，它们提供了强大的工具来创建符合Python习惯用法的、高度可定制的类和对象。通过这些机制，开发者可以充分利用Python的灵活性，编写出更加简洁、高效和Pythonic的代码。

#### 3.python中关于魔术方法与类装饰器的面试题 

##### 面试题1 

面试题目：  
设计一个类装饰器`timeit`，它可以测量任何类中方法的执行时间。然后，使用这个装饰器来装饰一个类，并展示如何测量该类中某个方法的执行时间。

面试考题知识点：

 *  类装饰器
 *  魔术方法`__call__`
 *  测量代码执行时间

考题知识点简要讲解：  
类装饰器是一种函数，用于在不修改类定义的情况下增强类的功能。通过定义一个包含`__call__`魔术方法的类，可以创建一个可调用的对象，这个对象可以作为装饰器来使用。`__call__`方法允许一个类的实例表现得像函数一样，可以被调用。在本题中，我们将使用类装饰器来测量方法的执行时间，这涉及到在调用方法前后获取时间，从而计算出方法的运行时间。

答案或代码：


In [ ]:
import time

class timeit:
    def __init__(self, cls):
        self.cls = cls

    def __call__(self, *args, **kwargs):
        class WrappedClass:
            def __init__(self, *args, **kwargs):
                self.original_instance = self.cls(*args, **kwargs)

            def __getattribute__(self, name):
                try:
                    attr = super().__getattribute__('original_instance').__getattribute__(name)
                except AttributeError:
                    raise AttributeError(f"{name} not found")
                
                if callable(attr):
                    def timed(*args, **kwargs):
                        start_time = time.time()
                        result = attr(*args, **kwargs)
                        end_time = time.time()
                        print(f"Executing {attr.__name__} took {end_time - start_time} seconds.")
                        return result
                    return timed
                else:
                    return attr
        return WrappedClass(*args, **kwargs)

@timeit
class MyClass:
    def method_to_time(self):
        time.sleep(2)
        return "Method complete"

# 使用装饰器装饰的类
obj = MyClass()
print(obj.method_to_time())


答案或代码解析：  
这个示例中，我们定义了一个名为`timeit`的类装饰器，它接收一个类作为参数，并返回一个包装过的类。在这个包装类中，我们重写了`__getattribute__`魔术方法，以便在调用任何方法时自动测量其执行时间。如果检测到的属性是可调用的（即方法），我们将其包装在一个新的函数`timed`中，这个函数记录方法执行的开始和结束时间，从而计算出执行时间，并在执行原方法后返回结果。

通过使用`timeit`装饰器装饰`MyClass`类，我们可以测量`method_to_time`方法的执行时间。当调用这个方法时，控制台将输出方法执行所需的时间，然后返回方法的结果。这种方法允许我们在不侵入原始类定义的情况下，为类的方法添加执行时间测量的功能。

##### 面试题2 

面试题目：  
设计一个类`LoggerDecorator`，它作为一个类装饰器，可以自动记录任何被装饰类的方法调用信息（例如方法名和调用时间）。然后，展示如何使用这个装饰器装饰一个类，并自动记录其方法调用。

面试考题知识点：

 *  类装饰器
 *  动态修改类的行为
 *  使用`__call__`和`__getattribute__`魔术方法

考题知识点简要讲解：  
类装饰器允许我们在不直接修改原始类定义的情况下，为类添加新的行为或修改现有行为。通过实现一个包含`__call__`方法的类，我们可以创建一个可调用的对象，这个对象可以作为装饰器应用于其他类。装饰器可以返回一个新的类，这个类通过重写`__getattribute__`方法来拦截对原始类方法的所有访问，从而实现在方法调用前后执行额外的逻辑。

答案或代码：


In [ ]:
import time

class LoggerDecorator:
    def __init__(self, cls):
        self.cls = cls

    def __call__(self, *args, **kwargs):
        class Wrapped:
            def __init__(self, *args, **kwargs):
                self.original_instance = self.cls(*args, **kwargs)

            def __getattribute__(self, name):
                try:
                    attr = super().__getattribute__('original_instance').__getattribute__(name)
                except AttributeError:
                    super().__getattribute__(name)

                if callable(attr):
                    def wrapper(*args, **kwargs):
                        print(f"Calling method: {name} at {time.ctime()}")
                        result = attr(*args, **kwargs)
                        return result
                    return wrapper
                return attr
        return Wrapped(*args, **kwargs)

@LoggerDecorator
class TestClass:
    def method_one(self):
        print("Executing method_one")

    def method_two(self, param):
        print(f"Executing method_two with param: {param}")

# 使用装饰器装饰的类
test = TestClass()
test.method_one()
test.method_two("Hello")


答案或代码解析：  
在这个示例中，`LoggerDecorator`类作为一个类装饰器，它接收一个类`cls`作为参数，并返回一个内部类`Wrapped`的实例。`Wrapped`类通过重写`__getattribute__`方法来拦截对任何方法的访问。如果访问的属性是可调用的（即方法），则返回一个`wrapper`函数，该函数在原方法调用前打印方法名和当前时间，然后调用原方法。

通过将`LoggerDecorator`应用于`TestClass`类，我们可以自动记录`TestClass`实例的任何方法调用。当调用`method_one`或`method_two`时，控制台将显示调用信息，包括方法名和调用时间，然后执行原始方法。

这种方法允许开发者在不修改原始类定义的情况下，为类的方法调用添加日志记录功能，从而提高代码的可维护性和可监控性。

##### 面试题3 

面试题目：  
如何使用类装饰器为Python类的所有公共方法添加异常捕获功能，以便在方法抛出异常时能够记录异常信息并重新抛出？

面试考题知识点：

 *  类装饰器的实现和应用
 *  动态地修改类的方法
 *  异常处理和记录

考题知识点简要讲解：  
类装饰器可以用来动态地修改或增强类的行为。在本例中，类装饰器的目标是为类的所有公共方法（即不以双下划线开头的方法）添加异常捕获逻辑。这可以通过在装饰器中创建一个新的包装类，并在这个包装类中使用`__getattribute__`魔术方法来拦截方法调用并添加异常处理逻辑来实现。当被装饰的类的方法被调用时，任何抛出的异常都将被捕获，异常信息将被记录，然后异常将被重新抛出。

答案或代码：


In [ ]:
import functools

def catch_exceptions(cls):
    class WrappedClass(cls):
        def __getattribute__(self, name):
            attr = super().__getattribute__(name)
            if not name.startswith("__") and callable(attr):
                @functools.wraps(attr)
                def wrapper(*args, **kwargs):
                    try:
                        return attr(*args, **kwargs)
                    except Exception as e:
                        print(f"Exception caught in {name}: {e}")
                        raise
                return wrapper
            return attr
    return WrappedClass

@catch_exceptions
class ExampleClass:
    def method(self, x):
        if x == 0:
            raise ValueError("Invalid value")
        return x

# 使用装饰器装饰的类
example = ExampleClass()
try:
    example.method(0)
except ValueError as e:
    print(f"Exception re-raised: {e}")


答案或代码解析：  
在这个示例中，我们定义了一个名为`catch_exceptions`的类装饰器，它接收一个类`cls`作为参数，并返回一个新的`WrappedClass`类。`WrappedClass`通过重写`__getattribute__`方法来拦截对类方法的访问。如果访问的属性是一个公共方法，我们使用`functools.wraps`装饰器来包装这个方法，添加一个`try...except`块来捕获并记录任何异常，然后重新抛出。

通过将`catch_exceptions`装饰器应用于`ExampleClass`，我们可以自动捕获并记录该类中所有公共方法抛出的异常。当`ExampleClass`的`method`方法因为无效的参数值而抛出`ValueError`时，异常信息将被记录，并且异常将被重新抛出，如示例中所见。

这种方法允许开发者为类的方法添加统一的异常处理逻辑，而无需修改每个方法的实现。它提高了代码的健壮性和可维护性，并有助于调试和错误追踪。

##### 面试题4 

面试题目：  
如何使用类装饰器和魔术方法`__getattribute__`来实现一个只读类，该类的任何属性一旦设置就不能被修改？

面试考题知识点：

 *  类装饰器的使用
 *  `__getattribute__`魔术方法的用法
 *  实现只读属性

考题知识点简要讲解：  
类装饰器用于在不修改类本身定义的情况下，动态地添加或修改类的行为。`__getattribute__`是一个魔术方法，它在访问任何属性时都会被调用，可以用来拦截对实例属性的所有访问。通过在类装饰器中使用`__getattribute__`，我们可以检查属性是否已经被设置，如果已经设置，则阻止修改，从而实现只读属性的效果。

答案或代码：


In [ ]:
def readonly(cls):
    class ReadOnlyWrapper:
        def __init__(self, *args, **kwargs):
            self._instance = cls(*args, **kwargs)
            self._frozen = False

        def __getattribute__(self, item):
            if item == "_frozen" or not self._frozen:
                return object.__getattribute__(self, item)
            raise AttributeError("This is a read-only attribute")

        def __setattr__(self, key, value):
            if self._frozen and hasattr(self, key):
                raise AttributeError("Cannot modify read-only attribute")
            object.__setattr__(self, key, value)
            if not key.startswith("_"):
                self._frozen = True

    return ReadOnlyWrapper

@readonly
class MyClass:
    def __init__(self, value):
        self.value = value

# 使用装饰器装饰的类
obj = MyClass(10)
print(obj.value)  # 输出: 10
obj.value = 20    # 抛出异常: AttributeError: Cannot modify read-only attribute


答案或代码解析：  
在这个示例中，`readonly`函数是一个类装饰器，它返回了一个新的`ReadOnlyWrapper`类。这个包装类的`__setattr__`方法会在尝试设置属性时被调用。如果实例的`_frozen`属性被设置为`True`，表示实例的属性已经被设置过，任何进一步的尝试修改属性都会导致抛出`AttributeError`。在第一次设置属性值之后，`_frozen`属性会被设置为`True`，从此实例变为只读。

通过将`readonly`装饰器应用于`MyClass`，我们创建了一个只读实例`obj`。在首次设置`value`属性后，任何尝试修改`value`属性的行为都将抛出异常，因为类的实例现在是只读的。

这个面试题展示了如何结合使用类装饰器和魔术方法来控制类的行为，实现只读属性，这在设计API和框架时尤其有用，可以防止消费者代码意外修改不应该被修改的属性。

##### 面试题5 

面试题目：  
如何使用类装饰器来确保一个类的所有公共方法都以单线程方式运行，即使在多线程环境中也是如此？

面试考题知识点：

 *  类装饰器的定义和使用
 *  线程同步机制，例如使用`threading.Lock`
 *  动态地修改类方法

考题知识点简要讲解：  
类装饰器可以用于在运行时修改或增强类的功能。在多线程环境中，为了保证线程安全，有时需要确保某些方法在同一时刻只能被一个线程调用。这可以通过在类装饰器中为每个公共方法添加线程锁来实现。`threading.Lock`提供了基本的线程同步机制，可以用来锁定方法，直到当前线程执行完毕后才释放，从而确保方法的线程安全。

答案或代码：


In [ ]:
import threading
from functools import wraps

def synchronized_methods(cls):
    lock = threading.Lock()

    for key in cls.__dict__:
        val = getattr(cls, key)
        if callable(val) and not key.startswith('_'):
            @wraps(val)
            def method(*args, lock=lock, original_method=val, **kwargs):
                with lock:
                    return original_method(*args, **kwargs)
            setattr(cls, key, method)
    return cls

@synchronized_methods
class ExampleClass:
    def __init__(self, value):
        self.value = value

    def increment(self):
        self.value += 1
        return self.value

    def decrement(self):
        self.value -= 1
        return self.value

# 使用装饰器装饰的类
example = ExampleClass(0)
print(example.increment())  # 输出: 1
print(example.decrement())  # 输出: 0


答案或代码解析：  
在这个示例中，`synchronized_methods`是一个类装饰器，它遍历类的所有属性，寻找可调用的公共方法（即不以`_`开头的方法）。对于每个这样的方法，它使用`wraps`装饰器来保持原始方法的元数据，然后创建一个新的方法，这个新方法会使用`threading.Lock`来确保在同一时刻只有一个线程可以执行这个方法。然后，它用同步版本的方法替换原始方法。

通过将`ExampleClass`类装饰为`synchronized_methods`，我们确保了`increment`和`decrement`方法在多线程环境下也是线程安全的。当我们创建`ExampleClass`的实例并调用这些方法时，由于使用了锁，即使在多线程环境中，这些方法也会被顺序执行。

这个面试题展示了如何使用类装饰器来增强类的线程安全性，这在设计多线程应用程序时尤其重要，可以避免由于并发访问共享资源而导致的数据竞争和不一致问题。

##### 面试题6 

面试题目：  
如何使用类装饰器和魔术方法`__setattr__`来创建一个类，该类能够自动验证其所有属性赋值是否符合特定的类型注解要求？

面试考题知识点：

 *  类装饰器的实现和应用
 *  `__setattr__`魔术方法的用法
 *  利用类型注解进行属性赋值验证

考题知识点简要讲解：  
类装饰器可以用来在不修改类定义的前提下增强类的功能。`__setattr__`是一个魔术方法，它在给属性赋值时被自动调用。通过结合类装饰器和`__setattr__`方法，我们可以在属性赋值时自动进行类型检查，确保赋值与类型注解相匹配。这样做可以在运行时提供类似静态类型语言的类型安全性。

答案或代码：


In [ ]:
def typechecked(cls):
    def validate(name, value, expected_type):
        if not isinstance(value, expected_type):
            raise TypeError(f"Expected type {expected_type} for attribute '{name}', got {type(value)}")

    original_setattr = cls.__setattr__

    def new_setattr(self, name, value):
        if name in cls.__annotations__:
            expected_type = cls.__annotations__[name]
            validate(name, value, expected_type)
        original_setattr(self, name, value)

    cls.__setattr__ = new_setattr
    return cls

@typechecked
class Person:
    name: str
    age: int

    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

# 使用装饰器装饰的类
person = Person("Alice", 30)  # 正确赋值
person.age = "thirty"         # 抛出异常: TypeError


答案或代码解析：  
在这个示例中，`typechecked`类装饰器首先保存了原始的`__setattr__`方法，然后定义了一个新的`new_setattr`方法来替换它。在`new_setattr`方法中，我们检查是否对任何有类型注解的属性进行了赋值。如果是，我们通过`isinstance`函数验证新值是否符合注解类型。如果不符合，抛出`TypeError`。然后，无论是否进行了类型检查，都使用原始的`__setattr__`方法设置属性值。

通过将`Person`类装饰为`typechecked`，我们确保了在对`name`和`age`属性赋值时进行了类型检查。当尝试将`age`属性设置为字符串`"thirty"`时，由于不符合类型注解`int`，装饰器抛出了`TypeError`。

这个面试题展示了如何使用类装饰器和魔术方法来实现运行时的类型检查，这在需要确保数据有效性和一致性的应用中非常有用。

##### 面试题7 

面试题目：  
如何使用类装饰器和魔术方法`__getattr__`来实现一个类，该类能够在访问未定义的属性时返回一个默认值？

面试考题知识点：

 *  类装饰器的实现
 *  `__getattr__`魔术方法的用法
 *  提供默认属性值

考题知识点简要讲解：  
类装饰器是一种函数，它用于在不修改类定义的情况下增强类的功能。`__getattr__`是一个魔术方法，它在访问的属性不存在时被调用。通过在类装饰器中使用`__getattr__`方法，我们可以捕获对不存在属性的访问，并提供一个默认值。这种方法可以用来创建具有更加灵活属性访问的类，例如在配置对象或开放式对象中非常有用。

答案或代码：


In [ ]:
def provide_default_attributes(cls):
    class ClassWithDefaults(cls):
        def __getattr__(self, item):
            if item not in self.__dict__:
                return "Default Value"  # 提供一个默认值
            return self.__dict__[item]

    return ClassWithDefaults

@provide_default_attributes
class MyClass:
    def __init__(self):
        self.defined_attribute = "I exist"

# 使用装饰器装饰的类
obj = MyClass()
print(obj.defined_attribute)  # 输出: I exist
print(obj.undefined_attribute)  # 输出: Default Value


答案或代码解析：  
这个示例中，`provide_default_attributes`函数是一个类装饰器，它创建并返回了一个新的类`ClassWithDefaults`。这个新类继承自原始类，并重写了`__getattr__`方法。当尝试访问一个未定义的属性时，`__getattr__`方法会被调用，并返回字符串`"Default Value"`作为默认值。

通过将`MyClass`类装饰为`provide_default_attributes`，我们为其实例`obj`提供了默认属性值的功能。尽管`obj`没有定义属性`undefined_attribute`，但当尝试访问它时，我们得到了默认值而非抛出`AttributeError`。

这个面试题展示了如何使用类装饰器和魔术方法来增强类的属性访问行为，使得类能够更加灵活地处理属性访问，这在某些应用场景下可以避免不必要的错误检查和属性初始化。

##### 面试题8 

面试题目：  
如何使用类装饰器结合`__slots__`魔术方法来限制类实例能够动态添加的属性？

面试考题知识点：

 *  类装饰器的定义和用法
 *  `__slots__`魔术方法的作用
 *  限制类实例属性

考题知识点简要讲解：  
类装饰器是用于修改或增强类的一种工具。`__slots__`是一个特殊的类属性，用于声明实例属性的固定集合，阻止自动创建`__dict__`和`__weakref__`属性，并且可以作为一种优化内存使用的方式。通过在类装饰器中添加`__slots__`属性，我们可以动态地为类添加这种行为，限制实例能够添加的属性集合，避免动态属性的创建。

答案或代码：


In [ ]:
def restrict_attributes(cls):
    cls.__slots__ = cls.__annotations__.keys()
    return cls

@restrict_attributes
class MyClass:
    name: str
    age: int

    def __init__(self, name, age):
        self.name = name
        self.age = age

# 使用装饰器装饰的类
obj = MyClass("John", 30)
print(obj.name)  # 输出: John
obj.new_attribute = "New Value"  # 将抛出 AttributeError


答案或代码解析：  
在这个示例中，`restrict_attributes`函数是一个类装饰器，它将类的`__annotations__`字典的键（即类型注解定义的属性名）赋值给`__slots__`。这样，`MyClass`类的实例只能拥有在`__slots__`中声明的属性，尝试添加任何其他属性都会导致抛出`AttributeError`。

通过将`MyClass`类装饰为`restrict_attributes`，我们限制了实例`obj`只能拥有`name`和`age`属性。尝试给`obj`添加一个未在`__slots__`中声明的新属性`new_attribute`时，Python解释器抛出了`AttributeError`，因为我们通过`__slots__`限制了属性的动态添加。

这个面试题展示了如何使用类装饰器来动态地为类添加`__slots__`属性，从而限制实例的属性并优化内存使用。这对于创建具有固定属性集的数据结构和防止用户错误地添加意外属性很有帮助。

##### 面试题9 

面试题目：  
如何使用类装饰器来实现单例模式，确保类只能创建一个实例？

面试考题知识点：

 *  单例模式的概念
 *  类装饰器的作用
 *  控制类实例的创建过程

考题知识点简要讲解：  
单例模式是一种常用的设计模式，它确保一个类只有一个实例，并提供一个全局访问点。在Python中，可以通过类装饰器来实现单例模式，装饰器可以拦截类的实例化过程。这通常涉及到捕获类的构造函数并在第一次调用时创建实例，然后在随后的调用中返回这个已创建的实例。

答案或代码：


In [ ]:
def singleton(cls):
    instances = {}

    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return get_instance

@singleton
class Database:
    def __init__(self):
        print("Loading database")

# 使用装饰器装饰的类
db1 = Database()  # 输出: Loading database
db2 = Database()  # 不会输出
print(db1 is db2)  # 输出: True


答案或代码解析：  
在这个示例中，`singleton`函数是一个类装饰器，它定义了一个`instances`字典来存储类的实例。装饰器返回的`get_instance`函数会在每次尝试实例化类时被调用。如果类的实例还没有被创建，它会创建一个新的实例并存储在`instances`字典中；如果实例已经存在，它将返回这个实例。

通过将`Database`类装饰为`singleton`，我们确保了无论尝试创建多少次`Database`的实例，都只会有一个实例被创建，并且每次都返回这个同一个实例。这可以通过检查`db1`和`db2`是否为同一个对象来验证，结果显示它们确实是相同的实例。

这个面试题展示了如何使用类装饰器来实现单例模式，这种模式在需要控制资源访问，如数据库连接或配置对象时非常有用。通过这种方式，可以确保全局只有一个实例被创建和使用。

##### 面试题10 

面试题目：  
如何使用类装饰器和魔术方法`__init_subclass__`来自动注册所有子类，在创建子类时无需显式注册？

面试考题知识点：

 *  类装饰器的实现
 *  `__init_subclass__`魔术方法的用法
 *  子类注册机制

考题知识点简要讲解：  
类装饰器可以用来修改或增强类的定义。`__init_subclass__`是Python 3.6中引入的一个魔术方法，它在创建一个类的子类时自动调用。我们可以利用这个方法来实现子类的自动注册，这样一来，任何时候继承了基类的子类都会被自动加入到一个注册表中，这在插件系统或需要基于子类类型进行动态操作的场景中非常有用。

答案或代码：


In [ ]:
def subclass_register(cls):
    cls._registry = {}

    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        cls._registry[cls.__name__] = cls

    cls.__init_subclass__ = classmethod(__init_subclass__)
    return cls

@subclass_register
class BasePlugin:
    pass

class PluginA(BasePlugin):
    pass

class PluginB(BasePlugin):
    pass

# 使用装饰器装饰的基类
print(BasePlugin._registry)  # 输出: {'PluginA': <class '__main__.PluginA'>, 'PluginB': <class '__main__.PluginB'>}


答案或代码解析：  
在这个示例中，`subclass_register`函数是一个类装饰器，它首先在基类中创建一个`_registry`字典用于存储子类。然后，它定义了`__init_subclass__`方法，并将其设置为类方法。每当有新的子类被定义时，`__init_subclass__`方法就会被调用，并将新的子类添加到`_registry`字典中。

通过将`BasePlugin`类装饰为`subclass_register`，我们实现了子类的自动注册。当`PluginA`和`PluginB`作为`BasePlugin`的子类被创建时，它们自动被添加到`BasePlugin._registry`中，这个注册表可以用来查找或实例化插件。

这个面试题展示了如何使用类装饰器和`__init_subclass__`魔术方法来自动化子类的注册过程，这在需要动态管理一组子类时特别有用，例如在框架开发或插件系统中。